In [1]:
#imports
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_internals import MahalanobisMonitor, GaussianMixtureMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor
from monitors_input import SHINE_monitor, SHINE_monitor2
from evaluation import Evaluator
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import utils

2022-07-26 07:14:23.103226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rsenaferre/anaconda3_new/envs/ANITI_RuntimeMonitoringBenchmark/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-26 07:14:23.103245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(x_train, y_train), (x_test, y_test) = utils.load_data('mnist', num_samples=None)
(ood_X_train, ood_y_train), (ood_X_test, ood_y_test) = utils.load_data('fashion_mnist', None)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
filename = 'models/sgd_mnist.sav'
scaler = StandardScaler()

#training ID dataset
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

nsamples, nx, ny, ndim = x_train.shape
train_dataset = x_train.reshape((nsamples,nx*ny*ndim))
x_train_scaled = scaler.fit_transform(train_dataset.astype(np.float64))

pred_train = loaded_model.predict(x_train_scaled)

print(accuracy_score(pred_train, y_train))

0.9014166666666666


In [4]:
#testing over ID test
print('testing ID...')
nsamples, nx, ny, ndim = x_test.shape
test_dataset = x_test.reshape((nsamples,nx*ny*ndim))

x_test_scaled = scaler.fit_transform(test_dataset.astype(np.float64))

pred_test = loaded_model.predict(x_test_scaled)

print(accuracy_score(pred_test, y_test))

testing ID...
0.8971


In [5]:
#testing over OOD test
filename = 'models/sgd_mnist.sav'
#scaler = StandardScaler()

print('testing OOD...')
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

nsamples, nx, ny, ndim = ood_X_test.shape
ood_test_dataset = ood_X_test.reshape((nsamples,nx*ny*ndim))

ood_x_test_scaled = scaler.fit_transform(ood_test_dataset.astype(np.float64))

pred_ood = loaded_model.predict(ood_x_test_scaled)

print(accuracy_score(pred_ood, ood_y_test))

testing OOD...
0.1474


In [ ]:
# internal values
print('features from the image', loaded_model.decision_function([x_test_scaled[0]]))

In [30]:
from scipy import spatial
import pickle
filename_c = 'Features/SGD/mnist_{}_correct.sav'
filename_i = 'Features/SGD/mnist_{}_incorrect.sav'

def similarity_act_func(features, id_y_train, pred_train):
    for cls in range(10):
        scores_1 = []
        scores_2 = []
        
        #all labels from class c
        ind_y_c = np.where(id_y_train == cls)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==cls)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #features from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        f_c_correct = features[list(ind)]

        #features from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        f_c_incorrect = features[list(ind)]

        # how similar are act functions between themselves?
        for i in f_c_incorrect:
            for c in f_c_correct:
                cosine_similarity = 1 - spatial.distance.cosine(c, i)
                scores_1.append(cosine_similarity)

        for i in range(1,len(f_c_correct)):
            cosine_similarity = 1 - spatial.distance.cosine(f_c_correct[i-1], f_c_correct[i])
            scores_2.append(cosine_similarity)
            
        pickle.dump(scores_1, open(filename_i.format(cls), 'wb'))
        pickle.dump(scores_2, open(filename_c.format(cls), 'wb'))
        print('sim correct/incorrect pred', np.sum(scores_1)/len(scores_1))
        print('sim between pairs de correct pred', np.sum(scores_2)/len(scores_2))

In [31]:
#calculating thresholds
#training set
#print('analysis for training set')
features_train = loaded_model.decision_function(x_train_scaled)
similarity_act_func(features_train, y_train, pred_train)


#test set
#print('analysis for test set')
features_test = loaded_model.decision_function(x_test_scaled)
#similarity_act_func(features_test, y_test, pred_test)

#OOD set
#print('analysis for OOD test set')
features_ood = loaded_model.decision_function(ood_x_test_scaled)
#similarity_act_func(features_ood, ood_y_test, pred_ood)

sim correct/incorrect pred 0.8802507817600072
sim between pairs de correct pred 0.9443786893853416
sim correct/incorrect pred 0.8761003266581424
sim between pairs de correct pred 0.9494050133214748
sim correct/incorrect pred 0.7754160302346598
sim between pairs de correct pred 0.8667444661828739
sim correct/incorrect pred 0.8236260199548552
sim between pairs de correct pred 0.8742108164545691
sim correct/incorrect pred 0.8171518240722214
sim between pairs de correct pred 0.8853001784424309
sim correct/incorrect pred 0.8036872757125182
sim between pairs de correct pred 0.8479743085284701
sim correct/incorrect pred 0.8452646012618847
sim between pairs de correct pred 0.9115823137034317
sim correct/incorrect pred 0.8422245282543268
sim between pairs de correct pred 0.912887384930604
sim correct/incorrect pred 0.7994836146220285
sim between pairs de correct pred 0.8690420363361099
sim correct/incorrect pred 0.8606543307953501
sim between pairs de correct pred 0.9142232165528579


In [7]:
#building SHINE monitor with ID data
id_dataset = 'mnist'
monitor_shine = SHINE_monitor(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
monitor_shine.fit_by_class(x_train, y_train, pred_train)

print('number of monitors',len(monitor_shine.arr_density))


#building SHINE 2 monitor with ID data
#monitor_shine2 = SHINE_monitor2(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
#monitor_shine2.fit_by_class(id_X_train, id_y_train, pred_train)

#print('number of monitors',len(monitor_shine2.arr_density))

{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
{'bandwidth': 1.0}
number of monitors 10


In [8]:
#same analysis but for SHINE scores

def shine_perf_analysis(id_X_train, id_y_training, pred_train, threshold_id):
    for c in range(10):
        scores_1 = []
        scores_2 = []
        #threshold_c = monitor_shine.scores_per_class[c]

        #all labels from class c
        ind_y_c = np.where(id_y_training == c)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==c)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #images from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        X_c_correct = id_X_train[list(ind)]

        #scores from correct
        for x, pred in zip(X_c_correct, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_1.append(monitor_pred)

        #images from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        X_c_incorrect = id_X_train[list(ind)]

        #scores from incorrect
        for x, pred in zip(X_c_incorrect, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_2.append(monitor_pred)

        print('avg score for correct pred', 1-(np.sum(scores_1)/len(scores_1)))
        print('avg score for incorrect pred', 1-(np.sum(scores_2)/len(scores_2)))

In [9]:
threshold_id = 0.9

#training set
print('training set analysis')
#shine_perf_analysis(id_X_train, id_y_train, pred_train, threshold_id)

#testing set
print('testing set analysis')
#shine_perf_analysis(id_X_test, id_y_test, pred_test, threshold_id)

#testing set
print('OOD test set analysis')
#shine_perf_analysis(ood_X_test, ood_y_test, pred_ood, threshold_id)

training set analysis
testing set analysis
OOD test set analysis


In [23]:
#hybrid version of SHINE
arr_ood_threshold = {}
arr_ood_threshold.update({0: 0.8802, 1: 0.8761, 2: 0.7754, 3: 0.8236, 4: 0.8171,
                          5:0.8036, 6:0.8452, 7:0.8422, 8:7994., 9:0.8606})

arr_id_threshold = {}
arr_id_threshold.update({0:0.9443, 1:0.9494, 2:0.8667, 3:0.8742, 4:0.8853,
                         5:0.8479, 6:0.9115, 7:0.9128, 8:0.8690, 9:0.9142})

#arr_id_threshold.update({0:0.94, 1:0.96, 2:0.94, 3:0.93, 4:0.96, 5:0.95, 6:0.94, 7:0.96, 8:0.96, 9:0.96})

def run_new_SHINE(monitor_shine, X, pred, incoming_feature, threshold_shine):
    features = features_train
    #all labels from class c
    ind_y_c = np.where(y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    avg_sim = np.sum(scores)/len(scores)
    
    if avg_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif avg_sim < arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_shine)
        
        #temporary
        if monitor_pred==True:
            monitor_pred=False
        elif monitor_pred==False:
            monitor_pred==True
        return monitor_pred

In [ ]:
## experimenting new thresholds for shine
def run_SHINE_2(monitor_shine, X, pred, incoming_feature, threshold_SMimg, threshold_SMout):
    features = features_train[id_layer_monitored]
    #all labels from class c
    ind_y_c = np.where(id_y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    len_scores = len(scores)
    sorted_scores = sorted(scores)
    ind_threshold = int(len_scores*threshold_SMout)
    min_threshold_sim = sorted_scores[-ind_threshold]
    max_threshold_sim = sorted_scores[ind_threshold]
    #avg_sim = np.sum(scores)/len(scores)
    
    if max_threshold_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif min_threshold_sim <= arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_SMimg)
        return monitor_pred

In [ ]:
#building oob monitor
monitor_oob = OutsideTheBoxMonitor(n_clusters=3)
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

In [24]:
#testing SHINE  and OOB monitors monitors on ID testset
m_true = []
m_shine = []
m_shine_2 = []
m_oob = []

threshold_SMimg = 0.1
threshold_SMout = 0.9

for x, pred, feature, label in tqdm(zip(x_test, pred_test, 
                                        features_test, y_test.flatten())):
    #SHINE
    monitor_shine_pred = run_new_SHINE(monitor_shine, np.array([x]), pred, feature, threshold_id)
    m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    #monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
    #                                   threshold_SMimg, threshold_SMout)
    #m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    #out_box = monitor_oob.predict([feature], [pred])
    #m_oob.append(out_box)
    
    if pred == label: #monitor does not need to activate
        m_true.append(0)
    else: #monitor should activate
        m_true.append(1)

10000it [20:07,  8.28it/s]


In [25]:
for x, feature, pred in tqdm(zip(ood_X_test, features_ood, pred_ood)):
    #SHINE
    monitor_shine_pred = run_new_SHINE(monitor_shine, np.array([x]), pred, feature, threshold_id)
    m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    #monitor_shine_pred_2 = run_SHINE_2(monitor_shine, np.array([x]), pred, feature,
    #                                   threshold_SMimg, threshold_SMout)
    #m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    #out_box = monitor_oob.predict([feature], [pred])
    #m_oob.append(out_box)
    
    m_true.append(1) #monitor should always react to novel classes

10000it [19:50,  8.40it/s]


In [26]:
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import balanced_accuracy_score

m_true = np.array(m_true).astype(bool)
m_shine = np.array(m_shine).astype(bool)
#m_shine_2 = np.array(m_shine_2).astype(bool)
#m_oob = np.array(m_oob).flatten().astype(bool)

#evaluating new SHINE 
print('\nSHINE new')
print(classification_report(m_true, m_shine))
print(mcc(m_true, m_shine))
print(balanced_accuracy_score(m_true, m_shine))

#evaluating new SHINE 2
#print('\nSHINE new 2')
#print(classification_report(m_true, m_shine_2))
#print(mcc(m_true, m_shine_2))
#print(balanced_accuracy_score(m_true, m_shine_2))

#evaluating OOB
#print('\nOOB')
#print(classification_report(m_true, m_oob))
#print(mcc(m_true, m_oob))
#print(balanced_accuracy_score(m_true, m_oob))


SHINE new
              precision    recall  f1-score   support

       False       0.61      0.88      0.72      8971
        True       0.85      0.54      0.66     11029

    accuracy                           0.69     20000
   macro avg       0.73      0.71      0.69     20000
weighted avg       0.74      0.69      0.69     20000

0.4401405353853451
0.7110052248326706


In [ ]:
'''
SHINE new
              precision    recall  f1-score   support

       False       0.63      0.69      0.66      8971
        True       0.73      0.67      0.69     11029

    accuracy                           0.68     20000
   macro avg       0.68      0.68      0.68     20000
weighted avg       0.68      0.68      0.68     20000

0.35391006402894293
0.6778852519809273
'''
'''
SHINE reverso
              precision    recall  f1-score   support

       False       0.61      0.88      0.72      8971
        True       0.85      0.54      0.66     11029

    accuracy                           0.69     20000
   macro avg       0.73      0.71      0.69     20000
weighted avg       0.74      0.69      0.69     20000

0.4401405353853451
0.7110052248326706
'''